# 0.   Packages and Functions

In [1]:
!pip install pandas_datareader

In [84]:

#general
import pandas as pd
import numpy as np
import pickle 
from decimal import Decimal


#for dates and times
import random
import time
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar

#prices
import pandas_datareader as web


In [3]:


def str_time_prop(start, end, format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(format, time.localtime(ptime))


def random_date(start, end, prop):
    return str_time_prop(start, end, '%Y-%m-%d', prop)

In [4]:
def weekend2weekday (transactions, col_date, col_day):


  dates2_BUY=list()
  days2_BUY=list()

  for index,row in transactions.iterrows():
    buy_date = datetime.datetime.strptime(row[col_date], "%Y-%m-%d")
    if row[col_day]=='Sat':
      buy_date2 = buy_date - datetime.timedelta(1)
      dates2_BUY.append(buy_date2)

      buy_day2='Fri'
      days2_BUY.append(buy_day2)
    elif row[col_day]=='Sun':
      buy_date = datetime.datetime.strptime(row[col_day], "%Y-%m-%d")
      buy_date2 = buy_date + datetime.timedelta(1)
      dates2_BUY.append(buy_date2)

      buy_day2='Mon'
      days2_BUY.append(buy_day2)
    else:
      buy_date2=row[col_date]
      dates2_BUY.append(buy_date2)
      buy_day2=row[col_day]
      days2_BUY.append(buy_day2)

    transactions['date_BUY_fix']=dates2_BUY
    transactions['day_BUY_fix']=days2_BUY


  return transactions

# 1. Setting investment parameters

## Temporal horizons
 In number of days
 - 1 day to 2 years

In [5]:
horizons=[1,2,3,4,5,6,7,10,15,20,30,40,50,60,70,80,90,100,120,150,180,210,240,270,300,330,360,420,480,540,600,660,720]


## Amounts in dollars
 - 50 to 50,000 dollars

In [6]:
amounts=[50,100,200,300,400,500,1000,1500,2000,3000,4000,5000,10000,15000,20000,25000,50000]

# 2. Data extraction

## Companies' stock prices

In [7]:
#Selecting a list of companies we are interested in analyzing:
companies = ['AAPL',
            'AMZN',
            'MSFT',
            'GOOG',
            'FB',
            'NVDA',
             #automobiles
            'F',
            'GM',
            'HMC',
            'TSLA',
            'TYO',
             'HOG',
             #retail
             'WMT',
             'M',
             'TGT',
             'KSS',
             'COST',
             'BBY',
             #FastMovingConsumerGoods(FMCG)
             'NESN',
             'KO',
             'PEP',
             'ULVR',
             'PG',
             'OR',
            #bank
            'AXP',
            'VISA',
            'SC',
            'DB',
            'UBS',
            'BAC',
            'JPM']

In [8]:
tech_c=['AAPL','AMZN','MSFT','GOOG','FB','VISA','NVDA']
auto_c=['F','GM','HMC','TSLA','TYO','HOG']
retail_c=[ 'WMT', 'M','TGT','KSS','COST','BBY']
fmcg_c=['NESN','KO','PEP','ULVR','PG', 'OR']
bank_c=['AXP','VISA','SC', 'DB', 'UBS','BAC','JPM']

In [9]:
#ESG ratios
ESG_ratios=[16.5, 27.4, 14.7, 22.8, 31.6, 12.9, 29.7, 30.6, 28.5, 31.1, 30.3, 16.3, 27.3, 14.5, 14.8, 12.9, 22.3, 12.0, 28.3, 25.1, 17.6, 23.4, 25.1, 20.5, 19.8, 17.4, 26.9, 30.0, 24.8, 26.3, 27.9]

We are going to extract info from 10 years from the companies selected

In [10]:
#Collecting the data of each company in a list
l_companies_prices=list()
error_companies=list()
error_indexes=list()

for company in companies:
    try:
        prices= web.DataReader(company, data_source='yahoo', start='2010-10-10', end='2020-10-10')
        l_companies_prices.append(prices)
        print(company)
    
    except :
        print('There were  errors when extracting data of  ', company)
        #elimino de la lista esa empresa que no se consiguieron datos.
        error_companies.append(company)
        
        error_company_index=companies.index(company)
        error_indexes.append(error_company_index)


for c in error_companies:
  companies.remove(c)

ESG_ratios_fix=list()

for index in range(0, len(ESG_ratios)):
    if index not in error_indexes:
        ESG_ratios_fix.append(ESG_ratios[index])

AAPL
AMZN
MSFT
GOOG
FB
NVDA
F
GM
HMC
TSLA
TYO
HOG
WMT
M
TGT
KSS
COST
BBY
There were  errors when extracting data of   NESN
KO
PEP
There were  errors when extracting data of   ULVR
PG
OR
AXP
There were  errors when extracting data of   VISA
SC
DB
UBS
BAC
JPM


In [11]:
companies

['AAPL',
 'AMZN',
 'MSFT',
 'GOOG',
 'FB',
 'NVDA',
 'F',
 'GM',
 'HMC',
 'TSLA',
 'TYO',
 'HOG',
 'WMT',
 'M',
 'TGT',
 'KSS',
 'COST',
 'BBY',
 'KO',
 'PEP',
 'PG',
 'OR',
 'AXP',
 'SC',
 'DB',
 'UBS',
 'BAC',
 'JPM']

In [12]:
len(ESG_ratios_fix)

28

In [13]:
len(companies)

28

In [14]:
print(companies[0])
l_companies_prices[0].columns

AAPL


Index(['High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close'], dtype='object')

In [15]:
company_pos=companies.index("AAPL")
l_companies_prices[company_pos].loc['2010-10-11','Close']

10.54857063293457

# 3. Random transactions

In [16]:
transactions=pd.DataFrame()
#we do 40,000 initial transactions

#id
transactions['id']=range(1,40001)

## Companies

In [17]:
#random company
transactions["company"]=np.random.choice(companies, size=len(transactions))

## Sector

In [18]:
#sector of the company

random_companies=transactions['company'].tolist()
random_sectors=list()
for comp in random_companies:
    if comp in tech_c:
        sector='TECH'
    elif comp in auto_c:
        sector='AUTO'
    elif comp in retail_c:
        sector='RETAIL'
    elif comp in fmcg_c:
        sector='FMCG'
    elif comp in bank_c:
        sector='BANK'
    
    random_sectors.append(sector)

transactions['sector']=random_sectors

## Horizons

In [19]:
#random horizon
transactions["horizon"] = np.random.choice(horizons, size=len(transactions))


## Investment Amounts

In [20]:
#random investment amount
transactions["amount"] = np.random.choice(amounts, size=len(transactions))


## Dates - BUY

In [21]:
#random buy date
start='2013-10-10'
end='2018-10-10'

dates_buy=list()

for i in range(0, transactions.shape[0]):
    r_date=random_date(start,end,random.random())
    dates_buy.append(r_date)


print(len(dates_buy))
transactions['date_BUY']=dates_buy

40000


### Fixing dates (BUY)

In [22]:
day_names=list()

for d in dates_buy:

  dt = datetime.datetime.strptime(d, '%Y-%m-%d')
  weekno=dt.weekday()

  if weekno == 0:
    dia='Mon'
    day_names.append(dia)
  elif weekno==1:
    dia='Tue'
    day_names.append(dia)
  elif weekno==2:
    dia='Wed'
    day_names.append(dia)
  elif weekno==3:
    dia='Thu'
    day_names.append(dia)
  elif weekno==4:
    dia='Fri'
    day_names.append(dia)
  elif weekno==5:
    dia='Sat'
    day_names.append(dia)
  elif weekno==6:
    dia='Sun'
    day_names.append(dia)

transactions['day_BUY']=day_names


In [23]:
dates2_BUY=list()
days2_BUY=list()

#if date is saturday, we are going to push it to friday
#if date is sunday, we are going to push it to monday

for index,row in transactions.iterrows():
  buy_date = datetime.datetime.strptime(row['date_BUY'], "%Y-%m-%d")
  if row['day_BUY']=='Sat':
    buy_date2 = buy_date - datetime.timedelta(1)
    dates2_BUY.append(buy_date2)

    buy_day2='Fri'
    days2_BUY.append(buy_day2)
  elif row['day_BUY']=='Sun':
    buy_date = datetime.datetime.strptime(row['date_BUY'], "%Y-%m-%d")
    buy_date2 = buy_date + datetime.timedelta(1)
    dates2_BUY.append(buy_date2)

    buy_day2='Mon'
    days2_BUY.append(buy_day2)
  else:
    buy_date2=buy_date
    dates2_BUY.append(buy_date2)
    buy_day2=row['day_BUY']
    days2_BUY.append(buy_day2)


transactions['date_BUY_fix']=dates2_BUY
transactions['day_BUY_fix']=days2_BUY

# transactions=weekend2weekday(transactions,'date_BUY','day_BUY' )

In [24]:
#checking for holidays (NYSE is closed)

cal = USFederalHolidayCalendar()
holidays = cal.holidays(start='2010-10-10', end='2020-10-10').to_pydatetime()
# if datetime.datetime(2014,01,01) in holidays:
#     print True 

l_holidays=list()
for d in dates2_BUY:
  if d in holidays:
    holiday='Holiday'
    l_holidays.append(holiday)

  else:
    holiday=''
    l_holidays.append(holiday)

transactions['holiday_BUY']=l_holidays

In [25]:
transactions_h=transactions[transactions.holiday_BUY=='Holiday']

transactions=transactions[transactions.holiday_BUY!='Holiday']

## Prices - BUY

In [26]:
#price when bought

l_buy_prices=list()
for index,row in transactions.iterrows():
  comp=row['company']
  buy_date=row['date_BUY_fix']
  company_pos=companies.index(comp)
  try:
    price=l_companies_prices[company_pos].loc[buy_date,'Adj Close']
  except :
    price=0

  l_buy_prices.append(price)

transactions['price_BUY']=l_buy_prices


## Dates - SELL

In [27]:
#random sell date
dates_SELL=list()

for index,row in transactions.iterrows():

  buy_date = row['date_BUY_fix']
  sell_date = buy_date + datetime.timedelta(row['horizon'])
  
  dates_SELL.append(sell_date)

transactions['date_SELL']=dates_SELL

### Fixing dates (SELL)

In [28]:
day_names=list()



for d in dates_SELL:

  # dt = datetime.datetime.strptime(d, '%Y-%m-%d')
  weekno=d.weekday()

  if weekno == 0:
    dia='Mon'
    day_names.append(dia)
  elif weekno==1:
    dia='Tue'
    day_names.append(dia)
  elif weekno==2:
    dia='Wed'
    day_names.append(dia)
  elif weekno==3:
    dia='Thu'
    day_names.append(dia)
  elif weekno==4:
    dia='Fri'
    day_names.append(dia)
  elif weekno==5:
    dia='Sat'
    day_names.append(dia)
  elif weekno==6:
    dia='Sun'
    day_names.append(dia)

transactions['day_SELL']=day_names


In [29]:
dates2_SELL=list()
days2_SELL=list()

#if date is saturday, we are going to push it to friday
#if date is sunday, we are going to push it to monday

for index,row in transactions.iterrows():
  SELL_date = row['date_SELL']
  if row['day_SELL']=='Sat':
    SELL_date2 = SELL_date - datetime.timedelta(1)
    dates2_SELL.append(SELL_date2)

    SELL_day2='Fri'
    days2_SELL.append(SELL_day2)
  elif row['day_SELL']=='Sun':
    SELL_date = row['date_SELL']
    SELL_date2 = SELL_date + datetime.timedelta(1)
    dates2_SELL.append(SELL_date2)

    SELL_day2='Mon'
    days2_SELL.append(SELL_day2)
  else:
    SELL_date2=SELL_date
    dates2_SELL.append(SELL_date2)
    SELL_day2=row['day_SELL']
    days2_SELL.append(SELL_day2)


transactions['date_SELL_fix']=dates2_SELL
transactions['day_SELL_fix']=days2_SELL

# transactions=weekend2weekday(transactions,'date_BUY','day_BUY' )

In [30]:
transactions

,id,company,sector,horizon,amount,date_BUY,day_BUY,date_BUY_fix,day_BUY_fix,holiday_BUY,price_BUY,date_SELL,day_SELL,date_SELL_fix,day_SELL_fix
0,1,COST,RETAIL,270,2000,2014-10-10,Fri,2014-10-10,Fri,,109.517578,2015-07-07,Tue,2015-07-07,Tue
1,2,AMZN,TECH,2,20000,2016-07-19,Tue,2016-07-19,Tue,,739.950012,2016-07-21,Thu,2016-07-21,Thu
2,3,DB,BANK,60,300,2018-09-29,Sat,2018-09-28,Fri,,11.166738,2018-11-27,Tue,2018-11-27,Tue
3,4,HOG,AUTO,90,200,2016-03-03,Thu,2016-03-03,Thu,,40.415287,2016-06-01,Wed,2016-06-01,Wed
4,5,WMT,RETAIL,10,200,2015-09-02,Wed,2015-09-02,Wed,,57.007484,2015-09-12,Sat,2015-09-11,Fri
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,39996,TYO,AUTO,100,500,2014-04-12,Sat,2014-04-11,Fri,,22.283308,2014-07-20,Sun,2014-07-21,Mon
39996,39997,TGT,RETAIL,80,50,2013-11-12,Tue,2013-11-12,Tue,,52.287945,2014-01-31,Fri,2014-01-31,Fri
39997,39998,NVDA,TECH,10,5000,2016-04-12,Tue,2016-04-12,Tue,,35.195301,2016-04-22,Fri,2016-04-22,Fri
39998,39999,F,AUTO,10,50000,2014-04-02,Wed,2014-04-02,Wed,,11.761689,2014-04-12,Sat,2014-04-11,Fri


In [31]:
#checking for holidays (NYSE, NASDAQ is closed)
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start='2010-10-10', end='2020-10-10').to_pydatetime()
# if datetime.datetime(2014,01,01) in holidays:
#     print True 

l_holidays=list()
for d in dates2_SELL:
  if d in holidays:
    holiday='Holiday'
    l_holidays.append(holiday)

  else:
    holiday=''
    l_holidays.append(holiday)

transactions['holiday_SELL']=l_holidays

## Prices - SELL

In [32]:
#price when sold

l_sell_prices=list()
for index,row in transactions.iterrows():
  comp=row['company']
  sell_date=row['date_SELL_fix']
  company_pos=companies.index(comp)
  try:
    price=l_companies_prices[company_pos].loc[sell_date,'Adj Close']
  except :
    price=0

  l_sell_prices.append(price)

transactions['price_SELL']=l_sell_prices


In [33]:
transactions_h=transactions[transactions.holiday_SELL=='Holiday']

transactions=transactions[transactions.holiday_SELL!='Holiday']

In [34]:
transactions

,id,company,sector,horizon,amount,date_BUY,day_BUY,date_BUY_fix,day_BUY_fix,holiday_BUY,price_BUY,date_SELL,day_SELL,date_SELL_fix,day_SELL_fix,holiday_SELL,price_SELL
0,1,COST,RETAIL,270,2000,2014-10-10,Fri,2014-10-10,Fri,,109.517578,2015-07-07,Tue,2015-07-07,Tue,,124.284370
1,2,AMZN,TECH,2,20000,2016-07-19,Tue,2016-07-19,Tue,,739.950012,2016-07-21,Thu,2016-07-21,Thu,,744.429993
2,3,DB,BANK,60,300,2018-09-29,Sat,2018-09-28,Fri,,11.166738,2018-11-27,Tue,2018-11-27,Tue,,9.672597
3,4,HOG,AUTO,90,200,2016-03-03,Thu,2016-03-03,Thu,,40.415287,2016-06-01,Wed,2016-06-01,Wed,,40.492554
4,5,WMT,RETAIL,10,200,2015-09-02,Wed,2015-09-02,Wed,,57.007484,2015-09-12,Sat,2015-09-11,Fri,,57.193264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,39996,TYO,AUTO,100,500,2014-04-12,Sat,2014-04-11,Fri,,22.283308,2014-07-20,Sun,2014-07-21,Mon,,20.795143
39996,39997,TGT,RETAIL,80,50,2013-11-12,Tue,2013-11-12,Tue,,52.287945,2014-01-31,Fri,2014-01-31,Fri,,45.549366
39997,39998,NVDA,TECH,10,5000,2016-04-12,Tue,2016-04-12,Tue,,35.195301,2016-04-22,Fri,2016-04-22,Fri,,35.617569
39998,39999,F,AUTO,10,50000,2014-04-02,Wed,2014-04-02,Wed,,11.761689,2014-04-12,Sat,2014-04-11,Fri,,11.168606


In [35]:
transactions_ready=transactions[['company','sector','horizon','amount','date_BUY_fix','date_SELL_fix','price_BUY','price_SELL']]

In [36]:
transactions_ready

,company,sector,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL
0,COST,RETAIL,270,2000,2014-10-10,2015-07-07,109.517578,124.284370
1,AMZN,TECH,2,20000,2016-07-19,2016-07-21,739.950012,744.429993
2,DB,BANK,60,300,2018-09-28,2018-11-27,11.166738,9.672597
3,HOG,AUTO,90,200,2016-03-03,2016-06-01,40.415287,40.492554
4,WMT,RETAIL,10,200,2015-09-02,2015-09-11,57.007484,57.193264
...,...,...,...,...,...,...,...,...
39995,TYO,AUTO,100,500,2014-04-11,2014-07-21,22.283308,20.795143
39996,TGT,RETAIL,80,50,2013-11-12,2014-01-31,52.287945,45.549366
39997,NVDA,TECH,10,5000,2016-04-12,2016-04-22,35.195301,35.617569
39998,F,AUTO,10,50000,2014-04-02,2014-04-11,11.761689,11.168606


## Filtering transactions with error (Price = 0)

In [37]:
price_error=transactions_ready[(transactions_ready.price_BUY==0) | (transactions_ready.price_SELL==0)]
print('with errors: ', price_error.shape)
print(transactions_ready.shape)

transactions_ready=pd.concat([transactions_ready, price_error, price_error]).drop_duplicates(keep=False)
print(transactions_ready.shape)

with errors:  (1472, 8)
(36229, 8)
(34683, 8)


# Inflacion
Buscamos que supere la tasa de inflacion

In [134]:
# # inflation=pd.read_csv('ratios_inflacion.csv', index_col=False)

# inflation['Year'] = pd.to_datetime(inflation['Year'])

# inflation=inflation.set_index('Year')

# # inflation.info()

In [143]:
# l_inflation=list()
# for index,row in transactions.iterrows():
#     buy_date=row['date_BUY_fix']
#     year=buy_date.year
#     print(year)
#     try:
#         inflation=inflation.loc[year,'Tasa_inflacion']
#     except :
#         inflation=0

#     l_inflation.append(inflation)

# transactions['inflation']=l_inflation


2014
2016
2018
2016
2015
2016
2017
2015
2017
2017
2014
2018
2017
2016
2017
2015
2017
2016
2017
2015
2014
2014
2017
2017
2018
2014
2018
2018
2018
2017
2017
2016
2016
2014
2017
2014
2016
2016
2015
2014
2015
2014
2013
2014
2016
2018
2014
2015
2014
2016
2018
2017
2013
2017
2017
2016
2015
2018
2017
2013
2014
2015
2013
2015
2018
2016
2016
2018
2015
2016
2015
2017
2014
2018
2014
2016
2017
2015
2014
2014
2017
2017
2016
2014
2017
2016
2018
2015
2018
2017
2015
2018
2017
2018
2017
2018
2015
2015
2016
2015
2014
2016
2015
2015
2018
2016
2017
2016
2017
2015
2016
2016
2015
2015
2016
2016
2017
2014
2015
2015
2015
2016
2015
2014
2015
2015
2017
2016
2015
2017
2017
2013
2018
2017
2014
2014
2013
2015
2017
2014
2014
2016
2017
2015
2013
2018
2016
2015
2018
2016
2016
2014
2014
2018
2018
2014
2016
2015
2018
2017
2015
2014
2017
2014
2015
2014
2016
2016
2016
2018
2016
2014
2016
2018
2014
2014
2015
2014
2017
2013
2017
2018
2018
2015
2018
2018
2017
2017
2013
2014
2014
2017
2015
2017
2018
2015
2014
2014
2018
2016


2017
2016
2016
2014
2014
2018
2013
2018
2017
2018
2015
2014
2016
2015
2016
2015
2018
2015
2015
2014
2017
2017
2016
2017
2016
2018
2015
2018
2014
2016
2018
2015
2018
2017
2015
2014
2018
2015
2016
2014
2014
2016
2017
2016
2015
2014
2017
2013
2014
2018
2016
2018
2014
2013
2015
2017
2015
2017
2018
2017
2014
2013
2018
2016
2015
2018
2015
2016
2015
2017
2018
2014
2018
2016
2016
2016
2015
2014
2015
2014
2016
2014
2015
2015
2017
2014
2015
2015
2016
2014
2017
2014
2014
2015
2014
2017
2016
2014
2016
2016
2017
2016
2018
2018
2016
2016
2018
2015
2014
2017
2014
2018
2018
2015
2016
2016
2018
2015
2018
2015
2017
2018
2018
2014
2016
2017
2017
2017
2018
2014
2018
2014
2017
2016
2014
2018
2015
2018
2014
2016
2014
2018
2014
2017
2014
2015
2018
2018
2015
2016
2014
2015
2015
2017
2015
2018
2015
2014
2015
2018
2014
2015
2017
2015
2014
2016
2015
2016
2018
2015
2017
2015
2014
2017
2017
2016
2017
2015
2015
2018
2017
2017
2014
2014
2014
2015
2014
2014
2017
2018
2017
2015
2014
2015
2017
2017
2017
2016
2015
2014


2018
2015
2016
2016
2017
2015
2015
2016
2016
2017
2015
2018
2015
2016
2016
2016
2017
2017
2015
2014
2016
2015
2018
2014
2016
2016
2014
2015
2016
2015
2018
2014
2015
2015
2014
2015
2015
2018
2016
2013
2016
2017
2014
2014
2016
2013
2013
2016
2016
2016
2015
2013
2014
2017
2015
2018
2014
2014
2017
2014
2014
2018
2017
2014
2016
2018
2013
2015
2017
2016
2015
2018
2016
2017
2014
2014
2015
2016
2015
2018
2014
2018
2017
2016
2017
2017
2015
2014
2017
2015
2015
2014
2015
2016
2018
2016
2016
2018
2015
2016
2014
2016
2017
2014
2014
2015
2016
2016
2014
2015
2017
2016
2015
2016
2015
2014
2014
2017
2013
2014
2014
2014
2016
2017
2018
2018
2016
2014
2017
2017
2016
2014
2015
2015
2018
2017
2017
2017
2014
2014
2015
2015
2014
2014
2015
2015
2015
2016
2015
2017
2014
2015
2017
2015
2015
2015
2017
2018
2017
2013
2018
2018
2014
2018
2017
2017
2017
2015
2014
2017
2015
2015
2017
2015
2017
2018
2015
2018
2016
2015
2015
2017
2017
2014
2014
2016
2015
2014
2018
2017
2018
2017
2018
2018
2017
2018
2014
2014
2014
2015


2016
2014
2017
2015
2014
2015
2013
2016
2014
2013
2015
2015
2017
2014
2017
2018
2014
2016
2017
2018
2017
2016
2014
2014
2018
2017
2016
2016
2014
2016
2015
2017
2017
2016
2014
2015
2016
2017
2016
2013
2018
2013
2016
2018
2014
2017
2017
2016
2017
2017
2013
2016
2018
2015
2016
2016
2014
2015
2015
2016
2018
2015
2014
2016
2014
2015
2016
2014
2014
2017
2014
2016
2017
2016
2017
2018
2017
2014
2018
2018
2014
2015
2016
2014
2018
2015
2014
2016
2016
2015
2018
2018
2016
2015
2018
2015
2017
2014
2014
2014
2016
2017
2016
2017
2017
2015
2015
2014
2018
2016
2017
2016
2016
2018
2016
2016
2014
2016
2018
2017
2018
2014
2014
2015
2016
2015
2013
2015
2014
2013
2013
2013
2015
2014
2017
2017
2017
2015
2017
2018
2017
2013
2018
2017
2014
2018
2015
2017
2018
2015
2017
2017
2015
2018
2014
2015
2017
2015
2018
2018
2015
2018
2014
2015
2015
2017
2015
2015
2018
2017
2014
2016
2016
2014
2017
2017
2015
2017
2014
2017
2017
2018
2017
2017
2016
2016
2015
2015
2014
2017
2014
2015
2016
2016
2017
2015
2016
2017
2017
2018


2014
2015
2014
2016
2016
2014
2014
2016
2018
2016
2013
2014
2016
2014
2013
2013
2014
2015
2015
2015
2015
2018
2015
2015
2016
2016
2016
2018
2016
2016
2018
2013
2017
2015
2017
2014
2014
2015
2016
2017
2018
2015
2017
2015
2017
2018
2016
2014
2017
2015
2015
2014
2015
2014
2016
2014
2014
2017
2014
2017
2014
2015
2015
2017
2016
2014
2015
2014
2014
2015
2017
2018
2014
2014
2015
2016
2017
2017
2014
2015
2018
2018
2014
2018
2015
2017
2015
2017
2014
2018
2017
2016
2016
2015
2017
2017
2017
2018
2016
2017
2018
2018
2017
2014
2016
2017
2015
2014
2017
2016
2014
2017
2014
2014
2015
2015
2017
2018
2016
2018
2015
2016
2016
2016
2016
2014
2015
2016
2015
2015
2017
2016
2014
2016
2016
2013
2015
2018
2015
2014
2015
2017
2015
2016
2014
2013
2015
2014
2016
2016
2015
2015
2014
2014
2017
2018
2015
2015
2014
2018
2013
2016
2013
2014
2018
2016
2017
2017
2016
2014
2016
2017
2015
2018
2013
2016
2015
2014
2016
2017
2016
2017
2014
2016
2018
2017
2016
2016
2018
2015
2014
2015
2014
2014
2017
2015
2016
2014
2017
2018


...

2018
2015
2017
2016
2016
2017
2017
2014
2015
2018
2014
2014
2014
2014
2017
2013
2014
2014
2016
2014
2016
2018
2017
2017
2016
2018
2015
2016
2014
2018
2018
2015
2016
2017
2014
2014
2018
2017
2014
2017
2017
2017
2015
2016
2014
2014
2017
2014
2016
2016
2017
2016
2013
2018
2018
2013
2015
2014
2015
2014
2018
2018
2018
2014
2014
2018
2017
2015
2014
2018
2017
2018
2016
2017
2017
2017
2015
2018
2016
2015
2014
2017
2017
2015
2014
2016
2018
2014
2014
2018
2015
2015
2016
2013
2016
2013
2017
2017
2017
2018
2016
2017
2014
2017
2017
2018
2016
2016
2018
2015
2014
2015
2017
2017
2014
2014
2017
2017
2017
2015
2014
2015
2015
2018
2016
2018
2017
2016
2014
2014
2014
2016
2018
2017
2014
2015
2016
2016
2018
2017
2016
2015
2013
2014
2017
2015
2014
2017
2017
2016
2015
2017
2013
2015
2016
2014
2014
2014
2015
2017
2017
2015
2017
2018
2014
2015
2018
2013
2018
2016
2014
2018
2016
2016
2018
2016
2017
2017
2016
2014
2018
2018
2017
2014
2018
2014
2017
2018
2016
2015
2015
2017
2017
2015
2016
2016
2015
2015
2016
2014


2017
2018
2016
2014
2014
2014
2014
2013
2016
2014
2013
2015
2014
2014
2014
2017
2014
2016
2017
2014
2013
2015
2018
2014
2017
2017
2016
2016
2016
2014
2015
2016
2017
2016
2016
2016
2014
2018
2018
2018
2016
2017
2014
2015
2014
2017
2017
2017
2015
2016
2017
2017
2016
2017
2014
2015
2016
2017
2017
2014
2017
2016
2017
2014
2016
2018
2014
2016
2015
2015
2015
2016
2014
2017
2016
2015
2014
2015
2014
2018
2017
2016
2016
2014
2016
2016
2016
2016
2017
2017
2017
2018
2017
2017
2016
2015
2017
2017
2016
2015
2014
2013
2014
2014
2015
2014
2014
2017
2017
2017
2015
2015
2014
2018
2018
2015
2016
2015
2014
2015
2018
2014
2017
2015
2014
2016
2015
2018
2017
2018
2018
2013
2018
2015
2013
2018
2016
2015
2014
2016
2016
2016
2013
2018
2014
2016
2017
2016
2014
2015
2016
2018
2017
2015
2014
2015
2018
2015
2015
2018
2014
2014
2017
2018
2014
2014
2018
2015
2015
2016
2017
2018
2014
2015
2014
2017
2016
2015
2017
2014
2016
2017
2015
2014
2016
2016
2015
2014
2018
2018
2015
2016
2016
2014
2016
2018
2015
2016
2014
2014


2014
2015
2017
2017
2016
2018
2014
2018
2016
2015
2015
2014
2017
2014
2017
2016
2017
2014
2017
2016
2016
2014
2015
2015
2015
2017
2016
2014
2018
2015
2016
2016
2014
2018
2014
2017
2017
2018
2014
2018
2018
2014
2015
2014
2018
2015
2018
2015
2014
2014
2018
2018
2014
2014
2013
2013
2014
2015
2014
2016
2018
2016
2015
2014
2016
2018
2013
2017
2017
2015
2018
2015
2015
2014
2018
2013
2017
2016
2017
2015
2015
2018
2014
2017
2014
2015
2017
2014
2014
2016
2014
2015
2017
2017
2015
2017
2014
2015
2018
2016
2017
2018
2018
2018
2016
2018
2015
2015
2015
2018
2016
2017
2015
2017
2016
2015
2016
2018
2015
2015
2016
2017
2013
2014
2017
2016
2017
2015
2016
2017
2018
2016
2018
2015
2017
2016
2017
2016
2017
2015
2017
2015
2014
2018
2015
2016
2018
2017
2014
2018
2016
2017
2017
2017
2013
2015
2014
2015
2014
2018
2017
2014
2016
2015
2017
2015
2016
2017
2014
2014
2017
2014
2016
2018
2014
2016
2013
2018
2018
2018
2014
2014
2016
2016
2017
2016
2016
2018
2018
2018
2014
2015
2014
2015
2017
2016
2016
2017
2014
2017


2015
2017
2015
2017
2016
2014
2015
2013
2014
2017
2015
2014
2017
2016
2017
2018
2017
2015
2017
2014
2016
2016
2014
2018
2015
2014
2016
2018
2015
2017
2014
2015
2017
2014
2014
2018
2014
2014
2016
2016
2017
2016
2015
2017
2016
2018
2016
2015
2015
2016
2014
2016
2014
2017
2017
2014
2014
2014
2016
2017
2014
2014
2014
2015
2016
2015
2018
2017
2014
2015
2014
2018
2017
2014
2017
2017
2014
2014
2015
2017
2016
2014
2017
2014
2015
2014
2017
2016
2016
2015
2017
2015
2016
2015
2017
2016
2017
2016
2015
2017
2015
2014
2017
2014
2016
2018
2015
2015
2014
2018
2017
2015
2016
2016
2016
2014
2016
2018
2017
2018
2016
2016
2014
2014
2016
2016
2015
2015
2016
2015
2016
2015
2018
2015
2015
2017
2017
2014
2015
2015
2015
2014
2014
2015
2018
2017
2016
2016
2016
2016
2017
2017
2013
2017
2017
2014
2014
2018
2014
2013
2013
2013
2016
2018
2015
2016
2018
2017
2018
2015
2018
2015
2014
2013
2017
2014
2016
2017
2017
2014
2017
2018
2016
2014
2016
2014
2014
2017
2016
2013
2016
2016
2013
2016
2013
2016
2015
2016
2014
2014


<ipython-input-143-20dbdd0dd96e>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions['inflation']=l_inflation


# Real Profit/Loss (Rentabilidad Real)

In [38]:
transactions_ready['nominal_return']=(transactions_ready['price_SELL']-transactions_ready['price_BUY'])/transactions_ready['price_BUY']

In [39]:
transactions_ready

,company,sector,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,nominal_return
0,COST,RETAIL,270,2000,2014-10-10,2015-07-07,109.517578,124.284370,0.134835
1,AMZN,TECH,2,20000,2016-07-19,2016-07-21,739.950012,744.429993,0.006054
2,DB,BANK,60,300,2018-09-28,2018-11-27,11.166738,9.672597,-0.133803
3,HOG,AUTO,90,200,2016-03-03,2016-06-01,40.415287,40.492554,0.001912
4,WMT,RETAIL,10,200,2015-09-02,2015-09-11,57.007484,57.193264,0.003259
...,...,...,...,...,...,...,...,...,...
39995,TYO,AUTO,100,500,2014-04-11,2014-07-21,22.283308,20.795143,-0.066784
39996,TGT,RETAIL,80,50,2013-11-12,2014-01-31,52.287945,45.549366,-0.128874
39997,NVDA,TECH,10,5000,2016-04-12,2016-04-22,35.195301,35.617569,0.011998
39998,F,AUTO,10,50000,2014-04-02,2014-04-11,11.761689,11.168606,-0.050425


In [40]:
transactions_ready['investment']=''
investments=list()
for index, row in transactions_ready.iterrows():
  if row['nominal_return']>0:
    inv='GOOD'
    investments.append(inv)
  elif row['nominal_return']<0:
    inv='BAD'
    investments.append(inv)
  else:
    inv='0'
    investments.append(inv)

transactions_ready['investment']=investments

In [41]:
transactions_ready.shape

(34683, 10)

In [42]:
transactions_ready

,company,sector,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,nominal_return,investment
0,COST,RETAIL,270,2000,2014-10-10,2015-07-07,109.517578,124.284370,0.134835,GOOD
1,AMZN,TECH,2,20000,2016-07-19,2016-07-21,739.950012,744.429993,0.006054,GOOD
2,DB,BANK,60,300,2018-09-28,2018-11-27,11.166738,9.672597,-0.133803,BAD
3,HOG,AUTO,90,200,2016-03-03,2016-06-01,40.415287,40.492554,0.001912,GOOD
4,WMT,RETAIL,10,200,2015-09-02,2015-09-11,57.007484,57.193264,0.003259,GOOD
...,...,...,...,...,...,...,...,...,...,...
39995,TYO,AUTO,100,500,2014-04-11,2014-07-21,22.283308,20.795143,-0.066784,BAD
39996,TGT,RETAIL,80,50,2013-11-12,2014-01-31,52.287945,45.549366,-0.128874,BAD
39997,NVDA,TECH,10,5000,2016-04-12,2016-04-22,35.195301,35.617569,0.011998,GOOD
39998,F,AUTO,10,50000,2014-04-02,2014-04-11,11.761689,11.168606,-0.050425,BAD


In [43]:
transactions_ready.investment.value_counts()

GOOD    20222
BAD     14052
0         409
Name: investment, dtype: int64

## Variables calculation

### Expected return

In [44]:
l_returns=list()
l_stdDev=list()

for company in companies:
  company_pos=companies.index(company)
  prices_r=l_companies_prices[company_pos]['Adj Close']

  # Calculting log returns and std deviation
  percent_change = prices_r.pct_change()
  returns = np.log(1+percent_change)

  meanReturn_daily = returns.mean()
  stdDev_daily = returns.std()    

  meanReturn_year= meanReturn_daily * 365
  stdDev_year= stdDev_daily *stdDev_daily* 365

  l_returns.append(meanReturn_year)
  l_stdDev.append(stdDev_year)
  


In [45]:
companies_returns=list(zip(companies,l_returns))
companies_stddev=list(zip(companies,l_stdDev))

In [46]:
companies_returns

[('AAPL', 0.37006814429896073),
 ('AMZN', 0.4447562102652766),
 ('MSFT', 0.34874318892447237),
 ('GOOG', 0.2509879900028765),
 ('FB', 0.33424316922218367),
 ('NVDA', 0.582100040715823),
 ('F', -0.036749878404302924),
 ('GM', 0.028801730711424594),
 ('HMC', -0.05782888896173675),
 ('TSLA', 0.6779141428282532),
 ('TYO', -0.2306144640410261),
 ('HOG', 0.016042203169368604),
 ('WMT', 0.174738668786327),
 ('M', -0.1474254373640375),
 ('TGT', 0.20006492374571294),
 ('KSS', -0.08109459573535757),
 ('COST', 0.28972100435924225),
 ('BBY', 0.19485826232397122),
 ('KO', 0.12239288649463223),
 ('PEP', 0.1512073670209265),
 ('PG', 0.16589283819663966),
 ('OR', -0.00748706568595733),
 ('AXP', 0.1710617622201037),
 ('SC', -0.024354755713299828),
 ('DB', -0.23159903755060685),
 ('UBS', -0.056055549945980554),
 ('BAC', 0.11337527794846122),
 ('JPM', 0.17508913733547116)]

In [47]:
#prueba de modificacion desde jupyter notebook.

# Ratios

In [48]:
l_PE_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix'].year
    company=row['company']
    try:
        PE_ratio=PE_ratios.loc[year,company]
    except:
        PE_ratio=0
    
    l_PE_ratios.append(PE_ratio)
    

In [49]:
transactions_ready['PE_ratio']=l_PE_ratios

In [50]:
transactions_ready

,company,sector,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,nominal_return,investment,PE_ratio
0,COST,RETAIL,270,2000,2014-10-10,2015-07-07,109.517578,124.284370,0.134835,GOOD,0
1,AMZN,TECH,2,20000,2016-07-19,2016-07-21,739.950012,744.429993,0.006054,GOOD,0
2,DB,BANK,60,300,2018-09-28,2018-11-27,11.166738,9.672597,-0.133803,BAD,0
3,HOG,AUTO,90,200,2016-03-03,2016-06-01,40.415287,40.492554,0.001912,GOOD,0
4,WMT,RETAIL,10,200,2015-09-02,2015-09-11,57.007484,57.193264,0.003259,GOOD,0
...,...,...,...,...,...,...,...,...,...,...,...
39995,TYO,AUTO,100,500,2014-04-11,2014-07-21,22.283308,20.795143,-0.066784,BAD,0
39996,TGT,RETAIL,80,50,2013-11-12,2014-01-31,52.287945,45.549366,-0.128874,BAD,0
39997,NVDA,TECH,10,5000,2016-04-12,2016-04-22,35.195301,35.617569,0.011998,GOOD,0
39998,F,AUTO,10,50000,2014-04-02,2014-04-11,11.761689,11.168606,-0.050425,BAD,0


## ESG Ratios

In [51]:
len(ESG_ratios_fix)

28

In [52]:
companies[3]

'GOOG'

In [53]:
ESG_ratios_fix

[16.5,
 27.4,
 14.7,
 22.8,
 31.6,
 12.9,
 29.7,
 30.6,
 28.5,
 31.1,
 30.3,
 16.3,
 27.3,
 14.5,
 14.8,
 12.9,
 22.3,
 12.0,
 25.1,
 17.6,
 25.1,
 20.5,
 19.8,
 26.9,
 30.0,
 24.8,
 26.3,
 27.9]

In [54]:
transactions_ready['ESG_ranking']=0

for index, row in transactions_ready.iterrows():
    company=row['company']
    ind_comp=companies.index(company)
    ESG_ratio= ESG_ratios_fix[ind_comp]
    transactions_ready.loc[index, 'ESG_ranking']=ESG_ratio
            

In [55]:
transactions_ready

,company,sector,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,nominal_return,investment,PE_ratio,ESG_ranking
0,COST,RETAIL,270,2000,2014-10-10,2015-07-07,109.517578,124.284370,0.134835,GOOD,0,22.3
1,AMZN,TECH,2,20000,2016-07-19,2016-07-21,739.950012,744.429993,0.006054,GOOD,0,27.4
2,DB,BANK,60,300,2018-09-28,2018-11-27,11.166738,9.672597,-0.133803,BAD,0,30.0
3,HOG,AUTO,90,200,2016-03-03,2016-06-01,40.415287,40.492554,0.001912,GOOD,0,16.3
4,WMT,RETAIL,10,200,2015-09-02,2015-09-11,57.007484,57.193264,0.003259,GOOD,0,27.3
...,...,...,...,...,...,...,...,...,...,...,...,...
39995,TYO,AUTO,100,500,2014-04-11,2014-07-21,22.283308,20.795143,-0.066784,BAD,0,30.3
39996,TGT,RETAIL,80,50,2013-11-12,2014-01-31,52.287945,45.549366,-0.128874,BAD,0,14.8
39997,NVDA,TECH,10,5000,2016-04-12,2016-04-22,35.195301,35.617569,0.011998,GOOD,0,12.9
39998,F,AUTO,10,50000,2014-04-02,2014-04-11,11.761689,11.168606,-0.050425,BAD,0,29.7


In [56]:
transactions_ready.loc[4,'date_BUY_fix']

Timestamp('2015-09-02 00:00:00')

In [65]:
ratios_collection= pickle.load(open('ratios_collection.p', "rb" ))

In [79]:
valor=ratios_collection['AAPL'][ratios_collection['AAPL']['Date']=='2020-09'].PE_Ratio.values[0]
float(valor)

35.46

In [60]:
#getting nearest date before investment.
def nearest(items, pivot):
    return min([i for i in items if i < pivot], key=lambda x: abs(x - pivot))

In [64]:
nearest(df['Date'].dropna(),transactions_ready.loc[4,'date_BUY_fix'])

Timestamp('2015-09-01 00:00:00')

In [85]:
l_PE_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix']
    company=row['company']
    
    try:
        ratios_company=ratios_collection[company]
        date=nearest(ratios_company['Date'].dropna(),row['date_BUY_fix'])
    
        try:
            PE_ratio_str=ratios_collection[company][ratios_collection[company]['Date']==date].PE_Ratio.values[0]
            PE_ratio=float(PE_ratio_str)
        except:
            #se puede asignar un cero si hace falta.
            PE_ratio= Decimal('nan')
            
    except:
        PE_ratio=Decimal('nan')
    
    l_PE_ratios.append(PE_ratio)

In [86]:
transactions_ready['PE_ratio']=l_PE_ratios

In [87]:
transactions_ready

,company,sector,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,nominal_return,investment,PE_ratio,ESG_ranking
0,COST,RETAIL,270,2000,2014-10-10,2015-07-07,109.517578,124.284370,0.134835,GOOD,25.12,22.3
1,AMZN,TECH,2,20000,2016-07-19,2016-07-21,739.950012,744.429993,0.006054,GOOD,191.6,27.4
2,DB,BANK,60,300,2018-09-28,2018-11-27,11.166738,9.672597,-0.133803,BAD,0,30.0
3,HOG,AUTO,90,200,2016-03-03,2016-06-01,40.415287,40.492554,0.001912,GOOD,12.02,16.3
4,WMT,RETAIL,10,200,2015-09-02,2015-09-11,57.007484,57.193264,0.003259,GOOD,10.84,27.3
...,...,...,...,...,...,...,...,...,...,...,...,...
39995,TYO,AUTO,100,500,2014-04-11,2014-07-21,22.283308,20.795143,-0.066784,BAD,NaN,30.3
39996,TGT,RETAIL,80,50,2013-11-12,2014-01-31,52.287945,45.549366,-0.128874,BAD,13.88,14.8
39997,NVDA,TECH,10,5000,2016-04-12,2016-04-22,35.195301,35.617569,0.011998,GOOD,29.85,12.9
39998,F,AUTO,10,50000,2014-04-02,2014-04-11,11.761689,11.168606,-0.050425,BAD,7.01,29.7
